In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import os
import numpy as np

### CONFIGURAÇÕES INICIAIS

In [2]:
# Configurações de estilo melhoradas
plt.style.use('ggplot')
rcParams['font.family'] = 'Arial'
rcParams['figure.figsize'] = (15, 10)
rcParams['axes.titlesize'] = 16
rcParams['axes.labelsize'] = 14
sns.set_palette("husl")

### CARREGAMENTO E PREPARAÇÃO DOS DADOS

In [3]:
# Lista de colunas a serem carregadas
colunas = ['V0001', 'V0015', 'V0026', 'V0031', 'B001', 'C006', 'C008', 'C009', 'C011', 'C012', 
           'D00201', 'D008', 'D00901', 'D01301', 'D01303', 'D014', 'D015', 'E001', 'E002', 'E003', 
           'E004', 'E005', 'E006011', 'E008', 'E010010', 'E011', 'E014011', 'E01601', 'E01603', 
           'E01605', 'E01602', 'E01604', 'E017', 'E01801', 'E01802', 'E01804', 'E019', 'E027', 
           'F001011', 'F007011', 'F008011', 'VDF001', 'F016', 'G059', 'G060', 'G062', 'G032', 
           'I00102', 'I00103', 'I001031', 'C00301', 'I006', 'I004', 'I00401', 'I00402', 'I00403',
           'I012', 'J001', 'J00101', 'J002', 'J00402', 'J00404', 'J007', 'J00801', 'J01101', 'J060', 
           'M001', 'M00203', 'M00302', 'M011011', 'M011021', 'M011051', 'M011071', 'N00101', 'N004', 
           'N005', 'N008', 'N010', 'N011', 'N012', 'N016', 'P00103', 'P00104', 'P00403', 'P00404', 
           'P00901', 'P01001', 'P01101', 'P02001', 'P018', 'P02002', 'P02501', 'P02602', 'P02601', 
           'P027', 'P02801', 'P029', 'P034', 'P035', 'P03701', 'P03702', 'P04501', 'P04502', 'P050', 
           'P051', 'P052', 'P053', 'P05401', 'P05404', 'P05407', 'P05410', 'P05413', 'P05416', 'P05419', 
           'P06701', 'P068', 'Q00101', 'Q00201', 'Q02901', 'Q03001', 'Q05901', 'Q060', 'Q06306', 
           'Q06307', 'Q06308', 'Q06309', 'Q06310', 'Q06311', 'Q064', 'Q06506', 'Q06507', 'Q06508', 
           'Q06509', 'Q06601', 'Q067', 'Q068', 'Q11006', 'Q128', 'R010', 'R025', 'R028', 'R029', 
           'R030', 'R031', 'R032', 'R033', 'R034', 'R03601', 'R03607', 'R03608', 'R03610', 'S065', 
           'T001', 'H001', 'H002', 'H003', 'H004', 'H010']

In [4]:
#categorical = ['V0001', 'V0015', 'V0026', 'V0031', 'B001', 'C006', 'C009', 'C011', 'C012', 'D00901', 
#               'D014', 'D015', 'E006011', 'E008', 'E010010', 'E027', 'F001011', 'F007011', 'F008011', 
#               'VDF001', 'F016', 'G059', 'G060', 'G062', 'G032', 'I00102', 'I006', 'I004', 'I00401', 
#               'I00402', 'I00403', 'I012', 'J001', 'J00101', 'J002', 'J00402', 'J00404', 'J007', 
#              'J00801', 'J01101', 'J060', 'M001', 'M00302', 'M011011', 'M011021', 'M011051', 'M011071', 
#               'N005', 'N008', 'N010', 'N011', 'N012', 'N016', 'P02601', 'P027', 'P050', 'P051', 'P06701', 
#               'P068', 'Q00201', 'Q03001', 'Q060', 'Q06306', 'Q06506', 'Q06507', 'Q06508', 'Q06509', 
#               'Q06601', 'Q067', 'Q068', 'Q11006', 'R034', 'R03601', 'R03607', 'R03608', 'R03610', 'S065', 
#               'T001', 'H003', 'H004', 'H010']

#continuous = ['C00701', 'C00702', 'C00703', 'E01602', 'E01604', 'E017', 'E01802', 'E01804', 'E019', 
#              'P00104', 'P00404', 'P00901', 'P01001', 'P01101', 'P02001', 'P018', 'P019', 'P02002', 
#              'P02501', 'P02602', 'P02801', 'P029', 'P035', 'P03701', 'P03702', 'P04501', 'P04502', 
#              'P053', 'P05401', 'P05404', 'P05407', 'P05410', 'P05413', 'P05416', 'P05419', 'P058', 'Q064']

In [5]:
# Carregar dados principais
try:
    df = pd.read_csv('..\..\Bases\Criados\\base_doencas_cardiacas.csv', 
                    usecols=colunas)
except Exception as e:
    print(f"Erro ao ler o arquivo de dados: {str(e)}")
    exit()

# Carregar mapeamento de códigos para nomes
try:
    nomes_colunas = pd.read_csv('..\..\Bases\Originais\PNS_titulos.csv', 
                              sep=';', 
                              encoding='latin')
    nome_por_codigo = dict(zip(nomes_colunas.iloc[:, 0], nomes_colunas.iloc[:, 1]))
except Exception as e:
    print(f"Erro ao ler o arquivo de mapeamento: {str(e)}")
    exit()

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:11: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:11: SyntaxWarning: invalid escape sequence '\.'
C:\Users\llays\AppData\Local\Temp\ipykernel_6776\4269659785.py:3: SyntaxWarning: invalid escape sequence '\.'
  df = pd.read_csv('..\..\Bases\Criados\\base_doencas_cardiacas.csv',
C:\Users\llays\AppData\Local\Temp\ipykernel_6776\4269659785.py:11: SyntaxWarning: invalid escape sequence '\.'
  nomes_colunas = pd.read_csv('..\..\Bases\Originais\PNS_titulos.csv',


### FILTRAGEM DOS DADOS

In [6]:
# Códigos das regiões
nordeste = [21, 22, 23, 24, 25, 26, 27, 28, 29]
centro_oeste = [50, 51, 52, 53]

# Aplicar filtros
df_filtrado = df.dropna(subset=['Q06306'])

df_filtrado = df_filtrado[
    (df_filtrado['V0015'] == 1) & # Tipo da entrevista: Realizada (1)
    (df_filtrado['V0001'].isin(nordeste + centro_oeste)) & # Região nordeste e centro-oeste
    (~df_filtrado['C012'].isin([3, 9])) & # Informante do Módulo C válido
    (~df_filtrado['D015'].isin([3, 9])) & # Informante do Módulo D válido
    (~df_filtrado['E027'].isin([3, 9])) & # Informante do Módulo E válido
    (~df_filtrado['F016'].isin([3, 9])) & # Informante do Módulo F válido
    (~df_filtrado['G032'].isin([3, 9])) & # Informante do Módulo G válido
    (~df_filtrado['I012'].isin([3, 9])) & # Informante do Módulo I válido
    (~df_filtrado['J060'].isin([3, 9])) & # Informante do Módulo J válido
    (df_filtrado['M001'] == 1) & # Entrevista do adulto selecionado: Realizada (1)
    (~df_filtrado['M00302'].isin([2])) & # Informante é morador
    (df_filtrado['Q064'] >= 40)  # Pessoas com 40 anos ou mais
]

# Selecionar apenas colunas existentes
colunas_existentes = [col for col in colunas if col in df_filtrado.columns]
df_filtrado = df_filtrado[colunas_existentes]

### IDENTIFICAÇÃO DE DADOS FALTANTES

In [7]:
# Identificar colunas com dados faltantes
colunas_com_faltantes = df_filtrado.columns[df_filtrado.isnull().any()].tolist()

if not colunas_com_faltantes:
    print("Nenhuma coluna com dados faltantes encontrada.")
    exit()

# Ordenar colunas por quantidade de dados faltantes (decrescente)
colunas_com_faltantes = sorted(colunas_com_faltantes, 
                              key=lambda x: df_filtrado[x].isna().sum(), 
                              reverse=True)

### FUNÇÃO PARA GRÁFICOS RELATIVOS (100% STACKED)

In [8]:
def plot_stacked_missing_data(df, columns_with_missing, nome_por_codigo, save_dir='Imagens'):
    """
    Função para plotar gráficos de barras empilhadas (100%) mostrando a proporção de dados faltantes
    
    Parâmetros:
    - df: DataFrame com os dados
    - columns_with_missing: lista de colunas com dados faltantes
    - nome_por_codigo: dicionário de mapeamento código -> nome
    - save_dir: diretório para salvar os gráficos
    """
    
    os.makedirs(save_dir, exist_ok=True)
    
    # Dividir em 5 gráficos
    num_grupos = 5
    grupos = np.array_split(columns_with_missing, num_grupos)
    
    for i, grupo in enumerate(grupos, 1):
        plt.figure(figsize=(18, 12))
        
        # Preparar dados
        present = []
        missing = []
        labels = []
        
        for col in grupo:
            nome_desc = nome_por_codigo.get(col, col)
            labels.append(f"{col}\n{nome_desc[:25] + '...' if len(nome_desc) > 25 else nome_desc}")
            
            total = len(df[col])
            miss = df[col].isna().sum()
            pres = total - miss
            
            present.append(pres)
            missing.append(miss)
        
        # Converter para porcentagens
        total = np.array(present) + np.array(missing)
        present_pct = np.array(present) / total * 100
        missing_pct = np.array(missing) / total * 100
        
        # Plotar
        bars_present = plt.barh(labels, present_pct, color='#2ecc71', label='Presente')
        bars_missing = plt.barh(labels, missing_pct, left=present_pct, color='#e74c3c', label='Faltante')
        
        # Adicionar valores
        for bar_p, bar_m, val_p, val_m, pct_p, pct_m in zip(bars_present, bars_missing, present, missing, present_pct, missing_pct):
            if pct_p > 5:
                plt.text(bar_p.get_width()/2, bar_p.get_y() + bar_p.get_height()/2, 
                        f'{val_p:,}\n({pct_p:.1f}%)', 
                        ha='center', va='center', color='white', fontsize=10, fontweight='bold')
            
            if pct_m > 5:
                plt.text(bar_p.get_width() + bar_m.get_width()/2, bar_m.get_y() + bar_m.get_height()/2, 
                        f'{val_m:,}\n({pct_m:.1f}%)', 
                        ha='center', va='center', color='white', fontsize=10, fontweight='bold')
        
        # Configurações
        plt.title(f'Proporção de Dados Faltantes - Gráfico {i} de {len(grupos)}\n(Total de registros: {len(df):,})', 
                 pad=20, fontsize=16)
        plt.xlabel('Porcentagem (%)', fontsize=14)
        plt.xlim(0, 100)
        plt.grid(True, axis='x', alpha=0.3)
        plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.12), ncol=2, fontsize=12)
        plt.tight_layout()
        
        # Salvar
        save_path = os.path.join(save_dir, f'stacked_missing_plot_{i}_of_{len(grupos)}.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Gráfico salvo em: {save_path}")

### EXECUÇÃO DA ANÁLISE

In [9]:
print("\nPlotando gráficos de proporção de dados faltantes:")
plot_stacked_missing_data(df_filtrado, colunas_com_faltantes, nome_por_codigo)

print("\nAnálise concluída com sucesso!")


Plotando gráficos de proporção de dados faltantes:
Gráfico salvo em: Imagens\stacked_missing_plot_1_of_5.png
Gráfico salvo em: Imagens\stacked_missing_plot_2_of_5.png
Gráfico salvo em: Imagens\stacked_missing_plot_3_of_5.png
Gráfico salvo em: Imagens\stacked_missing_plot_4_of_5.png


) missing from font(s) Arial.Temp\ipykernel_6776\3179855664.py:65: UserWarning: Glyph 13 (
  plt.tight_layout()
) missing from font(s) Arial.Temp\ipykernel_6776\3179855664.py:69: UserWarning: Glyph 13 (
  plt.savefig(save_path, dpi=300, bbox_inches='tight')


Gráfico salvo em: Imagens\stacked_missing_plot_5_of_5.png

Análise concluída com sucesso!


### ANÁLISE DE DADOS FALTANTES

In [10]:
# Análise de dados faltantes
print("\n" + "="*70)
print("ANÁLISE COMPLETA DE DADOS FALTANTES")
print("="*70)

# 1. Colunas SEM dados faltantes
colunas_sem_faltantes = df.columns[~df.isnull().any()].tolist()
print(f"\nCOLUNAS SEM DADOS FALTANTES ({len(colunas_sem_faltantes)}/{len(df.columns)}):")
print("-"*50)
for i, coluna in enumerate(colunas_sem_faltantes, 1):
    print(f"{i}. {coluna}")

# 2. Colunas COM dados faltantes
dados_faltantes = df.isnull().sum()
colunas_com_faltantes = dados_faltantes[dados_faltantes > 0]

print(f"\nCOLUNAS COM DADOS FALTANTES ({len(colunas_com_faltantes)}/{len(df.columns)}):")
print("-"*50)
print(f"{'Coluna':<15} {'Faltantes':>10} {'% Faltantes':>15}")
print("-"*50)

for coluna, qtd in colunas_com_faltantes.items():
    percentual = (qtd / len(df)) * 100
    print(f"{coluna:<15} {qtd:>10,} {percentual:>14.2f}%")

# 3. Resumo estatístico
total_faltantes = dados_faltantes.sum()
percentual_total = (total_faltantes / (len(df) * len(df.columns))) * 100

print("\nRESUMO ESTATÍSTICO:")
print("-"*50)
print(f"Total de colunas analisadas: {len(df.columns)}")
print(f"Colunas sem dados faltantes: {len(colunas_sem_faltantes)}")
print(f"Colunas com dados faltantes: {len(colunas_com_faltantes)}")
print(f"Total de valores faltantes: {total_faltantes:,}")
print(f"Percentual de dados faltantes: {percentual_total:.2f}%")

# Salvar relatório completo
with open('relatorio_dados_faltantes.txt', 'w', encoding='utf-8') as f:
    f.write("RELATÓRIO COMPLETO DE DADOS FALTANTES\n")
    f.write("="*70 + "\n\n")
    
    f.write(f"COLUNAS SEM DADOS FALTANTES ({len(colunas_sem_faltantes)}/{len(df.columns)}):\n")
    f.write("-"*50 + "\n")
    for coluna in colunas_sem_faltantes:
        f.write(f"{coluna}\n")
    
    f.write("\n\nCOLUNAS COM DADOS FALTANTES:\n")
    f.write("-"*50 + "\n")
    f.write(f"{'Coluna':<15} {'Faltantes':>10} {'% Faltantes':>15}\n")
    f.write("-"*50 + "\n")
    for coluna, qtd in colunas_com_faltantes.items():
        percentual = (qtd / len(df)) * 100
        f.write(f"{coluna:<15} {qtd:>10,} {percentual:>14.2f}%\n")
    
    f.write("\nRESUMO ESTATÍSTICO:\n")
    f.write("-"*50 + "\n")
    f.write(f"Total de colunas analisadas: {len(df.columns)}\n")
    f.write(f"Colunas sem dados faltantes: {len(colunas_sem_faltantes)}\n")
    f.write(f"Colunas com dados faltantes: {len(colunas_com_faltantes)}\n")
    f.write(f"Total de valores faltantes: {total_faltantes:,}\n")
    f.write(f"Percentual de dados faltantes: {percentual_total:.2f}%\n")

print("\nRelatório completo salvo em 'relatorio_dados_faltantes.txt'")


ANÁLISE COMPLETA DE DADOS FALTANTES

COLUNAS SEM DADOS FALTANTES (59/158):
--------------------------------------------------
1. V0001
2. V0015
3. V0026
4. V0031
5. B001
6. C006
7. C008
8. C009
9. C011
10. C012
11. D00201
12. D015
13. E001
14. E027
15. F001011
16. F007011
17. F008011
18. VDF001
19. F016
20. G059
21. G032
22. I00102
23. C00301
24. J001
25. J00101
26. J002
27. J007
28. J01101
29. J060
30. M001
31. N00101
32. N010
33. N011
34. N012
35. N016
36. P00901
37. P01101
38. P02001
39. P018
40. P02002
41. P02501
42. P02602
43. P02601
44. P027
45. P034
46. P04501
47. P04502
48. P050
49. P06701
50. P068
51. Q00101
52. Q02901
53. Q05901
54. Q06306
55. Q068
56. Q11006
57. Q128
58. T001
59. H001

COLUNAS COM DADOS FALTANTES (99/158):
--------------------------------------------------
Coluna           Faltantes     % Faltantes
--------------------------------------------------
D008                   687           2.82%
D00901               3,454          14.16%
D01301               6,9